In [126]:
# dé-commenter cette ligne si nbtutor est installé
# %load_ext nbtutor

In [127]:
from callstats import CallRecorder
from sys import setrecursionlimit

In [ ]:
from IPython.core.display import HTML

def _set_css_style(css_file_path):
   """
   Read the custom CSS file and load it into Jupyter.
   Pass the file path to the CSS file.
   """

   styles = open(css_file_path, "r").read()
   s = '<style>%s</style>' % styles     
   return HTML(s)

_set_css_style('custom/custom.css')

<div style='width:20em'><img src='img/logo-igm.png'></div>
<div style='font-size:larger'><strong>Algorithmique et programmation 2</strong><br>
L1 Mathématiques - L1 Informatique<br>
Semestre 2
</div>

# Récursivité et listes

1. Un type de données récursif : les listes chaînées
2. Fonctions récursives sur les listes chaînées
3. Récursivité et listes Python

## Un type de données récursif : les listes chaînées

### Rappel : définition récursive d'une fonction

On distingue un ou plusieurs cas "de base" et un cas "général"

Factorielle :

$$
n! = \begin{cases}
      1 \text{ si $n = 0$}\\
      n \times (n-1)! \text{ sinon}
    \end{cases}
$$

### Nouveauté : définir un type récursivement

**Idée :** une **liste récursive** (ou **liste chaînée**) est
- Soit vide (elle ne contient aucun élément)
- Soit constituée d'un premier élément et de la **liste** des éléments suivants

On définit l'ensemble des listes en faisant référence... à l'ensemble des listes

On appelle ces listes des **listes (simplement) chaînées** :

```mermaid
flowchart LR
    subgraph maillon1
    val1[12]
    ref1["⏺"]
    end
    subgraph maillon2
    99
    ref2["⏺"]
    end
    ref1 --> maillon2
    subgraph maillon3
    37
    ref3["⏺"]
    end
    ref2 --> maillon3
    ref3 --> vide["liste vide"]
```    

### En Python

On peut choisir de représenter :

- La liste vide par `None`
- La liste constituée du premier élément `tete` suivi de la liste d'éléments `suite` par le tuple `(tete, suite)`

**Remarque importante :** on représente ici des listes **immutables** !

#### Exemples :

In [129]:
vide = None  # liste à 0 éléments

In [130]:
nombres = (12, (99, (37, None)))

In [ ]:
saisons = ('printemps', 
              ('été', 
                  ('automne', 
                      ('hiver', None))))

**Attention :** ce ne sont pas **du tout** les listes habituelles de Python ! 

Pour éviter la confusion on distinguera
- **"listes chaînées"** ou **"listes récursives"**
- **"listes Python"** ou **"listes classiques"** ou **"listes natives"**

### Fonctions utilitaires

In [ ]:
from doctest import testmod
testmod()

In [133]:
def liste_vide():
    """
    Renvoie une liste récursive vide.
    """
    return None

In [134]:
def est_vide(r_liste):
    """
    Renvoie True si r_liste est vide, False sinon.

    >>> est_vide(liste_vide())
    True
    >>> est_vide(('a', ('b', ('c', None))))
    False
    """
    return r_liste == None

In [135]:
def ajout_debut(v, r_liste):
    """
    Crée une liste récursive contenant v suivi des éléments de r_liste.

    >>> ajout_debut('a', liste_vide())
    ('a', None)
    >>> ajout_debut('a', ajout_debut('b', liste_vide()))
    ('a', ('b', None))
    """
    return (v, r_liste)

In [ ]:
ajout_debut('a', 
    ajout_debut('b', 
        ajout_debut('c', 
            liste_vide())))

In [139]:
def tete(r_liste):
    """
    Renvoie l'élément de tête de r_liste.

    ATTENTION : plante si r_liste est vide !

    >>> tete(('a', ('b', None)))
    'a'

    >>> tete(None)
    Traceback (most recent call last):
    ...
    TypeError: 'NoneType' object is not subscriptable
    """
    return r_liste[0]

In [140]:
def suite(r_liste):
    """
    Renvoie r_liste privée de son élément de tête.

    ATTENTION : plante si r_liste est vide !

    >>> suite(('a', ('b', None)))
    ('b', None)

    >>> suite(None)
    Traceback (most recent call last):
    ...
    TypeError: 'NoneType' object is not subscriptable
    """
    return r_liste[1]

In [ ]:
nombres = (12, (99, (37, None)))
tete(nombres)

In [ ]:
suite(nombres)

## Parcours (récursif) de listes (récursives)

-   **Idée :** traiter un élément, puis appel récursif sur le reste, en déduire un résultat
-   **Cas d'arrêt :** liste vide, ou liste réduite à un élément

<img src="img/reclistes.png" width=70%></img>

Nombreuses variantes possibles *(dans l'autre sens, deux éléments à la fois...)*  

### Calculer la longueur d'une liste récursive

#### Algorithme

La longueur d'une liste `r_liste` est égale à :
- 0 si `r_liste` est vide
- la longueur de la liste `suite` plus 1 si `r_liste` est de la forme `(tete, suite)`

*(à écrire avec : `est_vide`, `tete`, `suite`)*

#### Implémentation

In [148]:
@CallRecorder
def longueur(r_liste):
    """
    Renvoie le nombre d'éléments de r_liste.

    >>> longueur(liste_vide())
    0
    >>> longueur(('a', ('b', ('c', None))))
    3
    """
    if est_vide(r_liste):
        return 0
    else:
        return 1 + longueur(suite(r_liste))

In [ ]:
vide = liste_vide()
longueur(vide)

In [ ]:
nombres = (12, (99, (37, None)))
longueur(nombres)
print(longueur(nombres).trace())

In [ ]:
saisons = ('printemps', ('été', ('automne', ('hiver', None))))
longueur(saisons)

### Rechercher un élément dans une liste récursive

Soit `r_liste` une liste récursive et `v` une valeur quelconque, on veut savoir si `v` apparaît dans `r_liste`.

#### Algorithme

- si `r_liste` est vide : c'est **faux**
- si `r_liste` est de la forme `(tete, suite)` avec `tete == v` : c'est **vrai**
- si `r_liste` est de la forme `(tete, suite)` avec `tete != v` : on recherche `v` dans `suite`

#### Implémentation

In [152]:
def est_dans(v, r_liste):
    """
    Renvoie True si v apparaît dans r_liste, False sinon.

    >>> est_dans('b', None)
    False
    >>> est_dans('b', ('a', ('b', ('c', None))))
    True
    >>> est_dans('d', ('a', ('b', ('c', None))))
    False
    """
    if est_vide(r_liste):
        return False
    elif tete(r_liste) == v:
        return True
    else:
        return est_dans(v, suite(r_liste))

In [ ]:
est_dans("automne", saisons)

In [ ]:
est_dans("juillet", saisons)

### Chercher la position d'un élément dans une liste récursive

Soit `r_liste` une liste récursive et `v` une valeur quelconque, on veut savoir à quelle position `v` apparaît pour la première fois dans `r_liste` (en comptant à partir de 0). 

On fixera par convention le résultat à `None` si la liste ne contient pas `v`, en particulier si la liste est vide.

#### Algorithme

Si `r_liste` est vide, le résultat est `None`. Dans le cas contraire, on suppose que `r_liste` est de la forme `(tete, suite)`.
- Si `tete == v` : le résultat est `0`
- Si `tete != v` : le résultat est 1 + la position de `v` dans `suite`

#### Implémentation

In [155]:
def premier_indice(v, r_liste):
    """
    Renvoie l'indice de première occurrence de c dans r_liste, ou None.
    
    >>> premier_indice('a', ('a', ('a', ('c', None))))
    0
    >>> premier_indice('c', ('a', ('a', ('c', None))))
    2
    >>> premier_indice('d', ('a', ('a', ('c', None)))) == None
    True
    """
    if est_vide(r_liste):
        return None
    elif tete(r_liste) == v:
        return 0
    else:
        res = premier_indice(v, suite(r_liste))
        if res is None: return None
        return 1 + res

In [ ]:
r_liste = (0, (1, (0, (1, (2, None)))))
premier_indice(2, r_liste)

In [ ]:
premier_indice("automne", saisons)

In [160]:
premier_indice("juillet", saisons)

In [167]:
def premier_indice_v2(v, r_liste, decalage=0):
    """
    Renvoie l'indice de première occurrence de c dans r_liste, ou None.
    
    >>> premier_indice_v2('a', ('a', ('a', ('c', None))), 0)
    0
    >>> premier_indice_v2('c', ('a', ('a', ('c', None))), 0)
    2
    >>> premier_indice_v2('d', ('a', ('a', ('c', None))), 0) == None
    True
    """
    if est_vide(r_liste):
        return None
    elif tete(r_liste) == v:
        return decalage
    else:
        return premier_indice_v2(v, suite(r_liste), decalage+1)

In [ ]:
premier_indice_v2(1, r_liste, 0)

In [ ]:
premier_indice_v2(1, r_liste)

In [ ]:
premier_indice_v2("automne", saisons, 10)

#### Variantes

##### Recherche du dernier indice où apparaît un élément

In [ ]:
def dernier_indice(v, r_liste):
    """
    Renvoie l'indice de dernière occurrence de c dans r_liste, ou None.
    
    >>> dernier_indice('a', ('a', ('a', ('c', None))))
    1
    >>> dernier_indice('c', ('a', ('a', ('c', None))))
    2
    >>> dernier_indice('d', ('a', ('a', ('c', None)))) == None
    True
    """
    if est_vide(r_liste):
        return None
    else:
        res = dernier_indice(v, suite(r_liste))
        if tete(r_liste) != v and res == None:
            return None
        elif res == None:
            return 0
        else:
            return 1 + res

In [ ]:
r_liste = (0, (1, (0, (1, (0, None)))))
dernier_indice(1, r_liste)

In [ ]:
dernier_indice(2, r_liste)

##### Liste des indices où apparaît un élément

In [ ]:
def liste_indices(v, r_liste):
    if est_vide(r_liste):
        return liste_vide()
    else:
        res = liste_indices(v, suite(r_liste))
        if tete(r_liste) == v:
            return ajout_debut(0, res)
        else:
            return res

In [ ]:
r_liste = (0, (1, (0, (1, (0, None)))))
liste_indices(0, r_liste)

##### Deuxième tentative

In [ ]:
def increment(r_liste):
    if est_vide(r_liste):
        return liste_vide()
    else:
        res = increment(suite(r_liste))
        return ajout_debut(tete(r_liste) + 1, res)

In [ ]:
increment((3, (12, (4, None))))

In [ ]:
def liste_indices(v, r_liste):
    if est_vide(r_liste):
        return liste_vide()
    else:
        res = liste_indices(v, suite(r_liste))
        res = increment(res)
        if tete(r_liste) == v:
            return ajout_debut(0, res)
        else:
            return res

In [ ]:
r_liste = (0, (1, (0, (1, (0, None)))))
liste_indices(0, r_liste)

##### Troisième tentative

In [ ]:
def liste_indices(v, r_liste, decalage):
    if est_vide(r_liste):
        return liste_vide()
    else:
        res = liste_indices(v, suite(r_liste), decalage+1)
        if tete(r_liste) == v:
            return ajout_debut(decalage, res)
        else:
            return res

In [ ]:
r_liste = (0, (1, (0, (1, (0, None)))))
liste_indices(0, r_liste, 0)

### Convertir une liste récursive en chaîne de caractères

... par exemple pour pouvoir l'afficher joliment.



#### Algorithme

- si la liste est vide, on renvoie une chaîne par défaut `"()"`
- sinon, on convertit la tête et (récursivement) la suite de la liste et on renvoie leur concaténation (en incluant un séparateur)

#### Implémentation

In [170]:
def liste_vers_chaine(r_liste):
    if est_vide(r_liste):
        return "()"
    else:
        return str(tete(r_liste)) + " -> " + liste_vers_chaine(suite(r_liste))

In [ ]:
liste_vers_chaine(nombres)

***Remarque :** Cette implémentation n'est pas très efficace, on verra pourquoi prochainement.*

### Renverser une liste chaînée

Il s'agit de construire une *nouvelle liste* contenant les mêmes éléments en ordre inverse. Cette opération est un peu plus difficile.

#### Algorithme
- si la liste est vide, on renvoie une liste vide
- sinon, on renverse la suite de la liste, et on y ajoute la tête comme dernier élément

#### Implémentation

*Problème :* on sait déjà ajouter un élément en début de liste avec `ajout_debut`, mais pas en fin — il faut une nouvelle fonction

In [172]:
def ajout_fin(v, r_liste):
    if est_vide(r_liste):
        return ajout_debut(v, r_liste)
    else:
        t, s = tete(r_liste), suite(r_liste)
        return ajout_debut(t, ajout_fin(v, s))

In [ ]:
nombres = (12, (99, (37, None)))
ajout_fin(5, nombres)

In [174]:
def renverser(r_liste):
    if est_vide(r_liste):
        return r_liste
    else:
        acc = renverser(suite(r_liste))
        return ajout_fin(tete(r_liste), acc)  # coûteux !

In [ ]:
renverser(nombres)

*Défaut :* Cette implémentation parcourt la liste entièrement à chaque nouvel élément à ajouter à `res` ! Cela peut vite devenir trop coûteux

#### Deuxième essai

In [176]:
def renverser2(r_liste):
    return renverser_aux(r_liste, liste_vide())

def renverser_aux(r_liste, acc):
    if est_vide(r_liste):
        return acc
    else:
        acc = ajout_debut(tete(r_liste), acc)
        return renverser_aux(suite(r_liste), acc)

In [ ]:
renverser2(nombres)

#### Commentaire sur la performance

Dans la seconde implémentation, on utilise seulement `ajout_debut`, qui ne parcourt pas la liste. Pour vérifier notre intuition sur la performance de ces deux variantes, on peut les chronométrer sur une liste suffisamment grande :

In [180]:
def premiers_entiers(n):
    acc = liste_vide()
    for i in range(n-1, -1, -1):
        acc = ajout_debut(i, acc)
    return acc

grande_liste = premiers_entiers(3000)

In [ ]:
%timeit renverser(grande_liste)
%timeit renverser2(grande_liste)

### Pour aller plus loin

- Toutes les fonctions montrées ci-dessus peuvent aussi être écrites facilement avec des boucles `while` (essayer !)
- Il reste à analyser la complexité des opérations sur les listes récursives, et la comparer avec les listes Python *(à venir dans un prochain épisode !)*
- On peut concevoir des structures de données récursives plus complexes (arbres, etc.)

## Récursivité sur les listes Python

**Question :** peut-on aussi écrire des fonctions récursives pour parcourir des listes Python ?

La réponse est bien sûr **oui**, mais avec quelques points de vigilance...

### Rappel : structure des listes Python

Les listes Python sont très différents des listes chaînées. Ce sont
- des **tableaux contigus** (rangées de « cases » les unes à côté des autres en mémoire)
- redimensionnés automatiquement selon le besoin
- programmés dans un langage de bas niveau, et très rapide

Il faut donc être très prudent quand on compare ces structures de données.

### Quelques exemples de fonctions récursives

Reprenons un exemple traité plus haut :

**Recherche** : Écrivez une fonction Python prenant en argument une liste et un objet quelconque, et renvoyant `True` si l'objet est dans la liste, `False` sinon... *sans utiliser de boucle ni le mot-clé `in`*.

#### Premier essai : avec une *slice*

In [ ]:
def est_dans_v1(elem, lst):
    if lst[0] == elem:
        return True
    else:
        return est_dans_v1(elem, lst[1:len(lst)])
    
est_dans_v1(0, [1, 2, 3])

*Problème :* oubli d'un cas d'arrêt

#### Second essai : *slice* et arrêt sur liste vide

In [ ]:
def est_dans_v2(elem, lst):
    if len(lst) == 0:
        return False
    elif lst[0] == elem:
        return True
    else:
        return est_dans_v2(elem, lst[1:])
        
est_dans_v2(0, [1, 2, 3])

*Problème :* calculer `lst[1:]` à chaque appel est **trop coûteux**

In [ ]:
setrecursionlimit(20000)
lst = [0] * 10**4
%timeit est_dans_v2(1, lst)

#### Troisième essai : paramètre supplémentaire

In [ ]:
def est_dans_v3(elem, lst, debut):
    if debut >= len(lst):
        return False
    elif lst[debut] == elem:
        return True
    else:
        return est_dans_v3(elem, lst, debut+1)  
    
lst = ['a', 'b', 'c']
est_dans_v3('d', lst, 0)

<img src="img/reclistes2.png" width=70%></img>

- Avantage : travaille toujours sur la même liste
- **Beaucoup** plus rapide

In [ ]:
lst = [0] * 10**4
%timeit est_dans_v3(1, lst, 0)

- Défaut : change la signature de la fonction (3 paramètres)

#### Variante : paramètre par défaut

In [ ]:
def est_dans_v4(elem, lst, debut=0):
    if debut >= len(lst):
        return False
    elif lst[debut] == elem:
        return True
    else:
        return est_dans_v4(elem, lst, debut+1)  
    
lst = ['a', 'b', 'c']
est_dans_v4('b', lst)

-   Avantage : travaille toujours sur la même liste
-   Valeur par défaut de `debut` si absent (`debut=0`)
-   Appel sans le troisième argument : cherche sur toute la liste

## Conclusion

### Pourquoi utiliser des structures de données récursives ?

- Parce qu'on n'a parfois pas le choix (et qu'il faut donc savoir le faire)
- Parce qu'elles interviennent sous une forme ou une autre dans de nombreux contextes en informatique


### Pourquoi programmer des fonctions récursives ?

-   On peut tout calculer de cette façon (sans boucles !)
-   Certains problèmes plus simples à résoudre
    
    -   Bien adapté à des structures de données récursives
    -   Solutions souvent plus concises et compréhensibles
    -   Algorithmes parfois plus faciles à analyser
    

-   Certains le sont beaucoup moins...
    
    -   Complexité parfois plus difficile à évaluer
    -   Opérations parfois inadaptées (ex : listes Python)
    
-   Peut entraîner un surcoût
    
    -   Temps de gestion de la pile
    -   Espace mémoire pour chaque appel imbriqué
    -   Recopies si on n'est pas prudent

### Bilan de la récursivité

-   Avantages:
    
    - $\checkmark$ simplifie la vie quand on a compris le ``truc''
    - $\checkmark$ code souvent plus court et donc plus lisible
    - $\checkmark$ principe fondamental en informatique
    
-   Inconvénients:

    - $\times$ il faut comprendre le truc...
    - $\times$ tous les langages ne s'y prêtent pas aussi bien
    - $\times$ souvent moins performant *($\rightarrow$ dérécursification, récursion terminale...)*

## <img src='img/non-exigible.png' style='display:inline; width:1.5em'> Raffinement du parcours de listes *(non exigible)*

**Problème :** 
- Le parcours récursif de listes Python vu précédemment recopie une portion de la liste à chaque appel (tous les éléments sauf 1) !
- C'est très coûteux en temps d'exécution et en mémoire.

In [ ]:
def est_dans_v2(lst, elem):
    if len(lst) == 0:
        return False
    elif lst[0] == elem:
        return True
    else:
        return est_dans_v2(lst[1:], elem)

est_dans_v2(list(range(10)), 14)

Cela revient (en plus rapide) à écrire une fonction `queue(lst)` qui renvoie une copie de `lst` privée de son premier élément :

In [ ]:
def queue(lst):
    res = []
    for i in range(1, len(lst)):
        res.append(lst[i])
    return res

def est_dans_2_bis(lst, elem):
    if len(lst) == 0:
        return False
    elif lst[0] == elem:
        return True
    else:
        return est_dans_2_bis(queue(lst), elem)
    
est_dans_2_bis(list(range(10)), 9)

In [ ]:
est_dans_2_bis([0, 0, 0, 0, 0], 1)

In [ ]:
est_dans_2_bis([0, 0, 0, 0, 1], 1)

**Exercice :** compter le nombre d'appels à la fonction `append`
-   au cours de l'appel `est_dans_2_bis([0, 0, 0, 0, 0], 1)`
-   au cours de l'appel `est_dans_2_bis([0] * n, 1)` pour `n` un naturel quelconque

In [ ]:
def queue(lst):
    res = []
    cpt = 0
    for i in range(1, len(lst)):
        res.append(lst[i])
        cpt += 1
    return res, cpt

def est_dans_2_ter(lst, elem):
    if len(lst) == 0:
        return False, 0
    elif lst[0] == elem:
        return True, 0
    else:
        tmp, cpt = queue(lst)
        res, cpt2 = est_dans_2_ter(tmp, elem)
        return res, cpt + cpt2        

In [ ]:
est_dans_2_ter([0, 0, 0, 0, 0], 1)

In [ ]:
est_dans_2_ter([0, 0, 0, 0, 1], 1)

In [ ]:
est_dans_2_ter([0]*1000, 1)

#### Troisième essai : avec `pop(0)`

In [ ]:
def est_dans_v3(lst, elem):
    if len(lst) == 0:
        return False 
    premier = lst.pop(0)
    if premier == elem:
        return True
    else:
        return est_dans_v3(lst, elem)

lst = [1, 2, 3]
print(est_dans_v3(lst, 3))
print(lst)
print(est_dans_v3(lst, 3))

*Avantage :* une seule copie, mais... 
- détruit la liste !
- toujours inefficace ! (`pop(0)` décale toute la liste)

#### Quatrième essai : avec `pop()`

In [ ]:
def est_dans_v4(lst, elem):
    if len(lst) == 0:
        return False 
    dernier = lst.pop()
    if dernier == elem:
        return True
    else:
        return est_dans_v4(lst, elem)
    

In [ ]:
lst = ['a', 'b', 'c']
est_dans_v4(lst, 'b')

In [ ]:
est_dans_v4(lst, 'b')

-   Parcourt la liste depuis la fin 
-   Plus efficace, mais... détruit toujours la liste !  

#### Cinquième essai : avec un indice

In [ ]:
def est_dans_v5(lst, elem, indice):
    if indice >= len(lst):
        return False
    elif lst[indice] == elem:
        return True
    else:
        return est_dans_v5(lst, elem, indice+1)  
    
lst = ['a', 'b', 'c']
est_dans_v5(lst, 'b', 0)

- Avantage : travaille toujours sur la même liste
- Défaut : change la signature de la fonction (3 paramètres)

#### Sixième essai : avec un paramètre par défaut

In [ ]:
def est_dans_v6(lst, elem, indice=0):
    if indice >= len(lst):
        return False
    elif lst[indice] == elem:
        return True
    else:
        return est_dans_v6(lst, elem, indice+1)
       

In [ ]:
lst = ['a', 'b', 'c']
est_dans_v6(lst, 'd')

-   Avantage : travaille toujours sur la même liste
-   Valeur par défaut de `indice` si absent (`indice=0`)
-   Appel sans le troisième argument : cherche sur toute la liste

#### Une dernière forme : avec une fonction interne

In [ ]:
def est_dans_v7(lst, elem):

    # fonction auxiliaire récursive!
    def aux(indice):  
        if indice >= len(lst):
            return False
        elif lst[indice] == elem:
            return True
        else:
            return aux(indice + 1)

    return aux(0)

lst = ['a', 'b', 'c']
est_dans_v7(lst, 'c')

**On compare !**

Pour voir un peu comment ces fonctions se comportent en pratique, on va les chronométrer avec `timeit` sur des entrées de la forme `[0] * n`.

In [ ]:
n = 20000

In [ ]:
setrecursionlimit(2*n)

In [ ]:
%timeit est_dans_v2([0] * n, 1)
%timeit est_dans_v3([0] * n, 1)
%timeit est_dans_v6([0] * n, 1)

Test de doublement :

In [ ]:
n = 10000
%timeit est_dans_v3([0] * n, 1)
%timeit est_dans_v3([0] * (n*2), 1)

Le temps n'est pas *du tout* proportionnel à la taille de l'entrée !

Logique : si $n (n-1) / 2 = x$, combien vaut $(2n) (2n-1) / 2$ ?

In [ ]:
n = 10000
%timeit est_dans_v6([0] * n, 1)
%timeit est_dans_v6([0] * (n*2), 1)

Le temps est *à peu près* proportionnel à la taille de l'entrée !